In [0]:
!pip install kaggle

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "sumeshmajee"
os.environ["KAGGLE_KEY"] = "KGAT_e8b80db00821caf2bf9e6a6690d004c4"

print("Kaggle credentials configured!")

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

In [0]:
%restart_python

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)

Day 1


In [0]:
# Create simple DataFrame
data = [("iPhone", 999), ("Samsung", 799), ("MacBook", 1299)]
df = spark.createDataFrame(data, ["product", "price"])
df.show()

# Filter expensive products
df.filter(df.price > 1000).show() 


In [0]:
import os
import getpass

# 1. Enter your token securely when prompted (It won't be saved in the file)
print("Paste your GitHub Token below and hit Enter:")
GITHUB_TOKEN = getpass.getpass()

# --- Everything else stays the same ---
USERNAME = "SRMajee"
REPO_NAME = "DataBricks_Challenge"
auth_url = f"https://{GITHUB_TOKEN}@github.com/{USERNAME}/{REPO_NAME}.git"

# 2. Update Remote
!git -c safe.directory='*' remote set-url origin {auth_url}

# 3. Add files (Now the file doesn't contain the secret!)
!git -c safe.directory='*' add .

# 4. Commit
!git -c safe.directory='*' commit -m "Secure push using getpass"

# 5. Push
!git -c safe.directory='*' push origin main